# Tool use, using langchain

This notebook demonstrates tool use, using langchain

In [1]:
#%pip install -qU langchain-google-vertexai py_expression_eval

In [1]:
from dotenv import load_dotenv

load_dotenv("../genai_agents/keys.env");

In [2]:
from langchain_core.tools import tool
from py_expression_eval import Parser
from langchain_google_vertexai import ChatVertexAI
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage

/opt/conda/lib/python3.10/site-packages/langgraph/graph/graph.py:30: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langgraph.pregel import Channel, Pregel


## 1. Create a tool. This tool will evaluate math formulas

In [3]:
@tool
def calculate(formula: str, parameters: dict) -> float:
    """Calculate by applying a mathematical formula to a set of input parameters"""
    parser = Parser()
    return parser.parse(formula).evaluate(parameters)

calculate.invoke({
    'formula': "(-b + sqrt(b^2 - 4*a*c))/(2*a)", 
    'parameters': {'a': 1.0, 'b': 1.0, 'c': -6.0}
})
#calculate("(-b + sqrt(b^2 - 4*a*c))/(2*a)", {'a': 1.0, 'b': 1.0, 'c': -6.0})

2.0

## 2. Use the LLM on a question that requires some math evaluation

In [10]:
model = ChatVertexAI(model="gemini-1.5-flash")
agent_executor = create_react_agent(model, [calculate])

prompt_message = HumanMessage(content="""
    You are a financial expert who can write down the appropriate formula and
    apply the mathematical formula to calculate the desired value.
    
    Question:
    If I invest $1000 in the stock market, and it returns 7% annually, and
    I reinvest the returns, how much money will I have in 6 years?
    
    Answer:
    """
)
response = agent_executor.invoke({"messages": [prompt_message]})

print(response)

{'messages': [HumanMessage(content='\n    You are a financial expert who can write down the appropriate formula and\n    apply the mathematical formula to calculate the desired value.\n    \n    Question:\n    If I invest $1000 in the stock market, and it returns 7% annually, and\n    I reinvest the returns, how much money will I have in 6 years?\n    \n    Answer:\n    ', additional_kwargs={}, response_metadata={}, id='0b358991-b6fa-464e-98d9-f735d6955f5a'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'calculate', 'arguments': '{"formula": "principal * (1 + rate) ^ years", "parameters": {"principal": 1000.0, "rate": 0.07, "years": 6.0}}'}}, response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False, 'severity': 'HARM_SEVERITY_NEGL

In [12]:
for message in response['messages']:
    print("*******", type(message))
    print(message.content)
    print(message.additional_kwargs)

******* <class 'langchain_core.messages.human.HumanMessage'>

    You are a financial expert who can write down the appropriate formula and
    apply the mathematical formula to calculate the desired value.
    
    Question:
    If I invest $1000 in the stock market, and it returns 7% annually, and
    I reinvest the returns, how much money will I have in 6 years?
    
    Answer:
    
{}
******* <class 'langchain_core.messages.ai.AIMessage'>

{'function_call': {'name': 'calculate', 'arguments': '{"formula": "principal * (1 + rate) ^ years", "parameters": {"principal": 1000.0, "rate": 0.07, "years": 6.0}}'}}
******* <class 'langchain_core.messages.tool.ToolMessage'>
1500.7303518490005
{}
******* <class 'langchain_core.messages.ai.AIMessage'>
You will have $1500.73 in 6 years. 

{}


**Analysis**

Note that the LLM created the formula:  <pre>principal * (1 + rate) ^ years</pre>
and invoked the tool <pre>calculate</pre> to get the exact answer, which is $1500.73.